In [1]:
!pip install transformers datasets torch accelerate


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
from transformers import Trainer, TrainingArguments


In [3]:
%%writefile stories.txt
Once upon a time, there was a peaceful village near a forest.
A young girl dreamed of becoming a great adventurer.
The king was worried about a mysterious enemy approaching.


Writing stories.txt


In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [5]:
dataset = load_dataset("text", data_files={"train": "stories.txt"})


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print(dataset["train"][0])


{'text': 'Once upon a time, there was a peaceful village near a forest.'}


In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=64
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./gpt2-results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    logging_steps=100,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=6, training_loss=3.072170893351237, metrics={'train_runtime': 27.5478, 'train_samples_per_second': 0.327, 'train_steps_per_second': 0.218, 'total_flos': 293953536000.0, 'train_loss': 3.072170893351237, 'epoch': 3.0})

In [16]:
prompt = "Once upon a time"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    inputs["input_ids"],
    max_length=100,
    do_sample=True,
    temperature=0.7,
    top_k=10
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, I was in a place where I could only see the stars.

And then, as I was walking, I heard a voice say, "The sky is falling in on us, and the sky is falling down in our path."
I was confused and was confused.
I looked at the sky. I was in. I looked at the sky.

"The sky is falling down in our path."

I looked at the sky, and there
